In [1]:
from bs4 import BeautifulSoup as bs
import requests
import lxml
import json

import sqlite3
import pandas as pd

In [2]:
# Manually parse html into table_array because Beautiful Soup was only seeing first 70 rows of table, even in raw local html file
line_number = 0
table_array = [[]]

with open("taxonomy_html.txt", encoding='utf-8') as f:
    for line in f:
        # Strings needed to find <font> tags
        start = '<font color="#595959">'
        end = "</font>"

        # Parse only text inside <font> tags into table_array
        if "</font></td>" in line or "</font></b></td>" in line or "</font></i></td>" in line:
            line_text = line[line.find(start)+len(start):line.rfind(end)].strip(' "')
            table_array[line_number].append(line_text)
        
        # Remove last two entries from row, data not needed
        if "</tr>" in line:
            table_array[line_number].pop()
            table_array[line_number].pop()
            line_number += 1
            table_array.append([])

# Remove final added empty list []
table_array.pop()


[]

In [3]:
# List of major dino groups we will be using
major_groups = ["Theropoda", "Sauropodomorpha", "Pachycephalosauria", "Stegosauria", "Ankylosauria", "Ceratopsia", "Ornithopoda"]

# Cleaned data array
group_array = []

# Loop through each entry and check if one of the major groups is present in the entry
for row in table_array:
    for item in row:
        for group in major_groups:
            if group in item:
                # Clean Genus/Species names
                sci_name = row[1].replace("&nbsp;", " ").replace("(1)", "").replace("(2)", "").replace("(3)", "")
                # Split the sci_name into Genus and Species if possible
                sci_name_split = sci_name.split()

                # Append the dinosaur's scientific name and its group to group_array, and ensure each entry in the array has 4 elements (leave species name blank if necessary)
                # Entry format: [SCI_NAME, GENUS, SPECIES, GROUP]
                if len(sci_name_split) == 1:
                    group_array.append([sci_name, sci_name_split[0], "", group])
                if len(sci_name_split) == 2:
                    group_array.append([sci_name, sci_name_split[0], sci_name_split[1], group])

In [4]:
with open('taxonomy_data.json', 'w', encoding='utf-8') as f:
    json.dump(group_array, f, ensure_ascii=False)